# Text mining on Non-Commercial Use Collection of OA Subset of PubMed 

In [6]:
import xml.etree.ElementTree as ET
import spacy
import os, sys,re
from html.parser import HTMLParser
import codecs
import fnmatch
nlp = spacy.load('en')
from geotext import GeoText
import nltk

# List of country names + iso codes 

countries = [
    ("Afghanistan", "AF", "AFG", "004", "Afghanistan"),
    ("Åland Islands", "AX", "ALA", "248", "Åland Islands"),
    ("Albania", "AL", "ALB", "008", "Albania"),
    ("Algeria", "DZ", "DZA", "012", "Algeria"),
    ("American Samoa", "AS", "ASM", "016", "American Samoa"),
    ("Andorra", "AD", "AND", "020", "Andorra"),
    ("Angola", "AO", "AGO", "024", "Angola"),
    ("Anguilla", "AI", "AIA", "660", "Anguilla"),
    ("Antarctica", "AQ", "ATA", "010", "Antarctica"),
    ("Antigua and Barbuda", "AG", "ATG", "028",
            "Antigua and Barbuda"),
    ("Argentina", "AR", "ARG", "032", "Argentina"),
    ("Armenia", "AM", "ARM", "051", "Armenia"),
    ("Aruba", "AW", "ABW", "533", "Aruba"),
    ("Australia", "A", "AUS", "036", "Australia"),
    ("Austria", "AT", "AUT", "040", "Austria"),
    ("Azerbaijan", "AZ", "AZE", "031", "Azerbaijan"),
    ("Bahamas", "BS", "BHS", "044", "Bahamas"),
    ("Bahrain", "BH", "BHR", "048", "Bahrain"),
    ("Bangladesh", "BD", "BGD", "050", "Bangladesh"),
    ("Barbados", "BB", "BRB", "052", "Barbados"),
    ("Belarus", "BY", "BLR", "112", "Belarus"),
    ("Belgium", "BE", "BEL", "056", "Belgium"),
    ("Belize", "BZ", "BLZ", "084", "Belize"),
    ("Benin", "BJ", "BEN", "204", "Benin"),
    ("Bermuda", "BM", "BM", "060", "Bermuda"),
    ("Bhutan", "BT", "BTN", "064", "Bhutan"),
    ("Bolivia, Plurinational State of", "BO", "BOL", "068",
            "Bolivia, Plurinational State of"),
    ("Bolivia", "BO", "BOL", "068",
            "Bolivia, Plurinational State of"),
    ("Bonaire, Sint Eustatius and Saba", "BQ", "BES", "535",
            "Bonaire, Sint Eustatius and Saba"),
    ("Bosnia and Herzegovina", "BA", "BIH", "070",
            "Bosnia and Herzegovina"),
    ("Bosnia", "BA", "BIH", "070",
            "Bosnia and Herzegovina"),
    ("Botswana", "BW", "BWA", "072", "Botswana"),
    ("Bouvet Island", "BV", "BVT", "074", "Bouvet Island"),
    ("Brazil", "BR", "BRA", "076", "Brazil"),
    ("British Indian Ocean Territory", "IO", "IOT", "086",
            "British Indian Ocean Territory"),
    ("Brunei Darussalam", "BN", "BRN", "096",
            "Brunei Darussalam"),
    ("Bulgaria", "BG", "BGR", "100", "Bulgaria"),
    ("Burkina Faso", "BF", "BFA", "854", "Burkina Faso"),
    ("Burundi", "BI", "BDI", "108", "Burundi"),
    ("Cambodia", "KH", "KHM", "116", "Cambodia"),
    ("Cameroon", "CM", "CMR", "120", "Cameroon"),
    ("Canada", "CA", "CAN", "124", "Canada"),
    ("Cabo Verde", "CV", "CPV", "132", "Cabo Verde"),
    ("Cayman Islands", "KY", "CYM", "136", "Cayman Islands"),
    ("Central African Republic", "CF", "CAF", "140",
            "Central African Republic"),
    ("Central Africa", "CF", "CAF", "140",
            "Central African Republic"),
    ("Chad", "TD", "TCD", "148", "Chad"),
    ("Chile", "CL", "CHL", "152", "Chile"),
    ("China", "CN", "CHN", "156", "China"),
    ("Christmas Island", "CX", "CXR", "162", "Christmas Island"),
    ("Cocos (Keeling) Islands", "CC", "CCK", "166",
            "Cocos (Keeling) Islands"),
    ("Cocos", "CC", "CCK", "166",
            "Cocos (Keeling) Islands"),
    ("Colombia", "CO", "COL", "170", "Colombia"),
    ("Comoros", "KM", "COM", "174", "Comoros"),
    ("Congo", "CG", "COG", "178", "Congo"),
    ("Congo, Democratic Republic of the", "CD", "COD", "180",
            "Congo, Democratic Republic of the"),
    ("Cook Islands", "CK", "COK", "184", "Cook Islands"),
    ("Costa Rica", "CR", "CRI", "188", "Costa Rica"),
    ("Côte d'Ivoire", "CI", "CIV", "384", "Côte d'Ivoire"),
    ("Croatia", "HR", "HRV", "191", "Croatia"),
    ("Cuba", "C", "CUB", "192", "Cuba"),
    ("Curaçao", "CW", "CUW", "531", "Curaçao"),
    ("Cyprus", "CY", "CYP", "196", "Cyprus"),
    ("Czechia", "CZ", "CZE", "203", "Czechia"),
    ("Denmark", "DK", "DNK", "208", "Denmark"),
    ("Djibouti", "DJ", "DJI", "262", "Djibouti"),
    ("Dominica", "DM", "DMA", "212", "Dominica"),
    ("Dominican Republic", "DO", "DOM", "214", "Dominican Republic"),
    ("Dominican", "DO", "DOM", "214", "Dominican Republic"),
    ("Ecuador", "EC", "EC", "218", "Ecuador"),
    ("Egypt", "EG", "EGY", "818", "Egypt"),
    ("El Salvador", "SV", "SLV", "222", "El Salvador"),
    ("Equatorial Guinea", "GQ", "GNQ", "226", "Equatorial Guinea"),
    ("Eritrea", "ER", "ERI", "232", "Eritrea"),
    ("Estonia", "EE", "EST", "233", "Estonia"),
    ("Ethiopia", "ET", "ETH", "231", "Ethiopia"),
    ("Falkland Islands (Malvinas)", "FK", "FLK", "238",
            "Falkland Islands (Malvinas)"),
    ("Faroe Islands", "FO", "FRO", "234", "Faroe Islands"),
    ("Fiji", "FJ", "FJI", "242", "Fiji"),
    ("Finland", "FI", "FIN", "246", "Finland"),
    ("France", "FR", "FRA", "250", "France"),
    ("French Guiana", "GF", "GUF", "254", "French Guiana"),
    ("French Polynesia", "PF", "PYF", "258", "French Polynesia"),
    ("French Southern Territories", "TF", "ATF", "260",
            "French Southern Territories"),
    ("Gabon", "GA", "GAB", "266", "Gabon"),
    ("Gambia", "GM", "GMB", "270", "Gambia"),
    ("Georgian", "GE", "GEO", "268", "Georgia"),
    ("Germany", "DE", "DEU", "276", "Germany"),
    ("Ghana", "GH", "GHA", "288", "Ghana"),
    ("Gibraltar", "GI", "GIB", "292", "Gibraltar"),
    ("Greece", "GR", "GRC", "300", "Greece"),
    ("Greenland", "GL", "GRL", "304", "Greenland"),
    ("Grenada", "GD", "GRD", "308", "Grenada"),
    ("Guadeloupe", "GP", "GLP", "312", "Guadeloupe"),
    ("Guam", "G", "GUM", "316", "Guam"),
    ("Guatemala", "GT", "GTM", "320", "Guatemala"),
    ("Guernsey", "GG", "GGY", "831", "Guernsey"),
    ("Guinea", "GN", "GIN", "324", "Guinea"),
    ("Guinea-Bissa", "GW", "GNB", "624", "Guinea-Bissa"),
    ("Guyana", "GY", "GUY", "328", "Guyana"),
    ("Haiti", "HT", "HTI", "332", "Haiti"),
    ("Heard Island and McDonald Islands", "HM", "HMD", "334",
            "Heard Island and McDonald Islands"),
    ("Holy See", "VA", "VAT", "336", "Holy See"),
    ("Honduras", "HN", "HND", "340", "Honduras"),
    ("Hong Kong", "HK", "HKG", "344", "Hong Kong"),
    ("Hungary", "H", "HUN", "348", "Hungary"),
    ("Iceland", "IS", "ISL", "352", "Iceland"),
    ("India", "IN", "IND", "356", "India"),
    ("Indonesia", "ID", "IDN", "360", "Indonesia"),
    ("Iran, Islamic Republic of", "IR", "IRN", "364",
            "Iran, Islamic Republic of"),
    ("Iran", "IR", "IRN", "364",
            "Iran, Islamic Republic of"),
    ("Iraq", "IQ", "IRQ", "368", "Iraq"),
    ("Ireland", "IE", "IRL", "372", "Ireland"),
    ("Isle of Man", "IM", "IMN", "833", "Isle of Man"),
    ("Israel", "IL", "ISR", "376", "Israel"),
    ("Italy", "IT", "ITA", "380", "Italy"),
    ("Jamaica", "JM", "JAM", "388", "Jamaica"),
    ("Japan", "JP", "JPN", "392", "Japan"),
    ("Jersey", "JE", "JEY", "832", "Jersey"),
    ("Jordan", "JO", "JOR", "400", "Jordan"),
    ("Kazakhstan", "KZ", "KAZ", "398", "Kazakhstan"),
    ("Kenya", "KE", "KEN", "404", "Kenya"),
    ("Kiribati", "KI", "KIR", "296", "Kiribati"),
    ("Korea, Democratic People's Republic of", "KP", "PRK", "408",
            "Korea, Democratic People's Republic of"),
    ("Korea, Republic of", "KR", "KOR", "410", "Korea, Republic of"),
    ("Korea", "KR", "KOR", "410", "Korea, Republic of"),
    ("Kosovo", "XK", "XKX", "000", "Kosovo"),
    ("Kuwait", "KW", "KWT", "414", "Kuwait"),
    ("Kyrgyzstan", "KG", "KGZ", "417", "Kyrgyzstan"),
    ("Lao People's Democratic Republic", "LA", "LAO", "418",
            "Lao People's Democratic Republic"),
    ("Latvia", "LV", "LVA", "428", "Latvia"),
    ("Lebanon", "LB", "LBN", "422", "Lebanon"),
    ("Lesotho", "LS", "LSO", "426", "Lesotho"),
    ("Liberia", "LR", "LBR", "430", "Liberia"),
    ("Libya", "LY", "LBY", "434", "Libya"),
    ("Liechtenstein", "LI", "LIE", "438", "Liechtenstein"),
    ("Lithuania", "LT", "LT", "440", "Lithuania"),
    ("Luxembourg", "L", "LUX", "442", "Luxembourg"),
    ("Macao", "MO", "MAC", "446", "Macao"),
    ("Macedonia, the former Yugoslav Republic of", "MK", "MKD", "807",
            "Macedonia, the former Yugoslav Republic of"),
    ("Macedonia", "MK", "MKD", "807",
            "Macedonia, the former Yugoslav Republic of"),
    ("Madagascar", "MG", "MDG", "450", "Madagascar"),
    ("Malawi", "MW", "MWI", "454", "Malawi"),
    ("Malaysia", "MY", "MYS", "458", "Malaysia"),
    ("Maldives", "MV", "MDV", "462", "Maldives"),
 
    ("Malta", "MT", "MLT", "470", "Malta"),
    ("Marshall Islands", "MH", "MHL", "584", "Marshall Islands"),
    ("Martinique", "MQ", "MTQ", "474", "Martinique"),
    ("Mauritania", "MR", "MRT", "478", "Mauritania"),
    ("Mauritius", "M", "MUS", "480", "Mauritius"),
    ("Mayotte", "YT", "MYT", "175", "Mayotte"),
    ("Mexico", "MX", "MEX", "484", "Mexico"),
    ("Micronesia, Federated States of", "FM", "FSM", "583",
            "Micronesia, Federated States of"),
    ("Micronesia", "FM", "FSM", "583",
            "Micronesia, Federated States of"),
    
    ("Moldova, Republic of", "MD", "MDA", "498",
            "Moldova, Republic of"),
    
    ("Moldova", "MD", "MDA", "498",
            "Moldova, Republic of"),
    ("Monaco", "MC", "MCO", "492", "Monaco"),
    ("Mongolia", "MN", "MNG", "496", "Mongolia"),
    ("Montenegro", "ME", "MNE", "499", "Montenegro"),
    ("Montserrat", "MS", "MSR", "500", "Montserrat"),
    ("Morocco", "MA", "MAR", "504", "Morocco"),
    ("Mozambique", "MZ", "MOZ", "508", "Mozambique"),
    ("Myanmar", "MM", "MMR", "104", "Myanmar"),
    ("Namibia", "NA", "NAM", "516", "Namibia"),
    ("Naur", "NR", "NR", "520", "Naur"),
    ("Nepal", "NP", "NPL", "524", "Nepal"),
    ("Netherlands", "NL", "NLD", "528", "Netherlands"),
    ("New Caledonia", "NC", "NCL", "540", "New Caledonia"),
    ("New Zealand", "NZ", "NZL", "554", "New Zealand"),
    ("Nicaragua", "NI", "NIC", "558", "Nicaragua"),
    ("Niger", "NE", "NER", "562", "Niger"),
    ("Nigeria", "NG", "NGA", "566", "Nigeria"),
    ("Niue", "N", "NIU", "570", "Niue"),
    ("Norfolk Island", "NF", "NFK", "574", "Norfolk Island"),
    ("Northern Mariana Islands", "MP", "MNP", "580",
            "Northern Mariana Islands"),
    ("Norway", "NO", "NOR", "578", "Norway"),
    ("Oman", "OM", "OMN", "512", "Oman"),
    ("Pakistan", "PK", "PAK", "586", "Pakistan"),
    ("Pala", "PW", "PLW", "585", "Pala"),
    ("Palestine, State of", "PS", "PSE", "275",
            "Palestine"),
    ("Panama", "PA", "PAN", "591", "Panama"),
    ("Papua New Guinea", "PG", "PNG", "598",
            "Papua New Guinea"),
    ("Paraguay", "PY", "PRY", "600", "Paraguay"),
    ("Per", "PE", "PER", "604", "Per"),
    ("Philippines", "PH", "PHL", "608", "Philippines"),
    ("Pitcairn", "PN", "PCN", "612", "Pitcairn"),
    ("Poland", "PL", "POL", "616", "Poland"),
    ("Portugal", "PT", "PRT", "620", "Portugal"),
    ("Puerto Rico", "PR", "PRI", "630", "Puerto Rico"),
    ("Qatar", "QA", "QAT", "634", "Qatar"),
    ("Réunion", "RE", "RE", "638", "Réunion"),
    ("Romania", "RO", "ROU", "642", "Romania"),
    ("Russian Federation", "R", "RUS", "643",
            "Russian Federation"),
    ("Russia", "R", "RUS", "643",
            "Russian Federation"),
    ("Rwanda", "RW", "RWA", "646", "Rwanda"),
    ("Saint Barthélemy", "BL", "BLM", "652",
            "Saint Barthélemy"),
    ("Saint Helena, Ascension and Tristan da Cunha",
            "SH", "SHN", "654",
            "Saint Helena, Ascension and Tristan da Cunha"),
    ("Saint Kitts and Nevis", "KN", "KNA", "659",
            "Saint Kitts and Nevis"),
    ("Saint Lucia", "LC", "LCA", "662", "Saint Lucia"),
    ("Saint Martin (French part)", "MF", "MAF", "663",
            "Saint Martin (French part)"),
    ("Saint Pierre and Miquelon", "PM", "SPM", "666",
            "Saint Pierre and Miquelon"),
    ("Saint Vincent and the Grenadines", "VC", "VCT", "670",
            "Saint Vincent and the Grenadines"),
    ("Samoa", "WS", "WSM", "882", "Samoa"),
    ("San Marino", "SM", "SMR", "674", "San Marino"),
    ("Sao Tome and Principe", "ST", "STP", "678",
            "Sao Tome and Principe"),
    ("Saudi Arabia", "SA", "SAU", "682", "Saudi Arabia"),
    ("Senegal", "SN", "SEN", "686", "Senegal"),
    ("Serbia", "RS", "SRB", "688", "Serbia"),
    ("Seychelles", "SC", "SYC", "690", "Seychelles"),
    ("Sierra Leone", "SL", "SLE", "694", "Sierra Leone"),
    ("Singapore", "SG", "SGP", "702", "Singapore"),
    ("Sint Maarten (Dutch part)", "SX", "SXM", "534",
            "Sint Maarten (Dutch part)"),
    ("Slovakia", "SK", "SVK", "703", "Slovakia"),
    ("Slovak Republic", "SK", "SVK", "703", "Slovakia"),
    ("Slovak", "SK", "SVK", "703", "Slovakia"),
    
    ("Slovenia", "SI", "SVN", "705", "Slovenia"),
    ("Solomon Islands", "SB", "SLB", "090", "Solomon Islands"),
    ("Somalia", "SO", "SOM", "706", "Somalia"),
    ("South Africa", "ZA", "ZAF", "710", "South Africa"),
    ("South Georgia and the South Sandwich Islands",
            "GS", "SGS", "239",
            "South Georgia and the South Sandwich Islands",),
    ("South Sudan", "SS", "SSD", "728", "South Sudan"),
    ("Spain", "ES", "ESP", "724", "Spain"),
    ("Sri Lanka", "LK", "LKA", "144", "Sri Lanka"),
    ("Sudan", "SD", "SDN", "729", "Sudan"),
    ("Suriname", "SR", "SUR", "740", "Suriname"),
    ("Svalbard and Jan Mayen", "SJ", "SJM", "744",
            "Svalbard and Jan Mayen"),
    ("Eswatini", "SZ", "SWZ", "748", "Eswatini"),
    ("Sweden", "SE", "SWE", "752", "Sweden"),
    ("Switzerland", "CH", "CHE", "756", "Switzerland"),
    ("Syrian Arab Republic", "SY", "SYR", "760",
            "Syrian Arab Republic"),
    ("Syrian", "SY", "SYR", "760",
            "Syrian Arab Republic"),

    ("Taiwan, Province of China", "TW", "TWN", "158",
            "Taiwan"),
    ("Taiwan", "TW", "TWN", "158",
            "Taiwan"),
    
    ("Tajikistan", "TJ", "TJK", "762", "Tajikistan"),
    ("Tanzania, United Republic of", "TZ", "TZA", "834",
            "Tanzania, United Republic of"),
    ("Thailand", "TH", "THA", "764", "Thailand"),
    ("Timor-Leste", "TL", "TLS", "626", "Timor-Leste"),
    ("Togo", "TG", "TGO", "768", "Togo"),
    ("Tokela", "TK", "TKL", "772", "Tokela"),
    ("Tonga", "TO", "TON", "776", "Tonga"),
    ("Trinidad and Tobago", "TT", "TTO", "780",
            "Trinidad and Tobago"),
    ("Tunisia", "TN", "TUN", "788", "Tunisia"),
    ("Turkey", "TR", "TUR", "792", "Turkey"),
    ("Turkmenistan", "TM", "TKM", "795", "Turkmenistan"),
    ("Turks and Caicos Islands", "TC", "TCA", "796",
            "Turks and Caicos Islands"),
    ("Tuval", "TV", "TUV", "798", "Tuval"),
    ("Uganda", "UG", "UGA", "800", "Uganda"),
    ("Ukraine", "UA", "UKR", "804", "Ukraine"),
    ("United Arab Emirates", "AE", "ARE", "784",
            "United Arab Emirates"),
    ("United Kingdom of Great Britain and Northern Ireland",
            "GB", "GBR", "826",
            "United Kingdom of Great Britain and Northern Ireland"),
    ("ENGLAND",
            "GB", "GBR", "826",
            "United Kingdom of Great Britain and Northern Ireland"),
    ("Scotland",
            "GB", "GBR", "826",
            "United Kingdom of Great Britain and Northern Ireland"),
    ("United Kingdom",
            "GB", "GBR", "826",
            "United Kingdom of Great Britain and Northern Ireland"),
    ("Wales",
            "GB", "GBR", "826",
            "United Kingdom of Great Britain and Northern Ireland"),
    ("Northern Ireland",
            "GB", "GBR", "826",
            "United Kingdom of Great Britain and Northern Ireland"),
    ("Great Britain",
            "GB", "GBR", "826",
            "United Kingdom of Great Britain and Northern Ireland"),
    ("UK",
            "GB", "GBR", "826",
            "United Kingdom of Great Britain and Northern Ireland"),
    
    ("United States of America", "US", "USA", "840",
            "United States of America"),
    ("United States", "US", "USA", "840",
            "United States of America"),
    ("USA", "US", "USA", "840",
            "United States of America"),
    ("United States Minor Outlying Islands", "UM", "UMI", "581",
            "United States Minor Outlying Islands"),
    ("Uruguay", "UY", "URY", "858", "Uruguay"),
    ("Uzbekistan", "UZ", "UZB", "860", "Uzbekistan"),
    ("Vanuat", "V", "VUT", "548", "Vanuat"),
    ("Venezuela, Bolivarian Republic of", "VE", "VEN", "862",
            "Venezuela, Bolivarian Republic of"),
    ("Venezuela", "VE", "VEN", "862",
            "Venezuela, Bolivarian Republic of"),
    ("Viet Nam", "VN", "VNM", "704", "Viet Nam"),
    ("Virgin Islands, British", "VG", "VGB", "092",
            "Virgin Islands, British"),
    ("Virgin Islands, U.S.", "VI", "VIR", "850",
            "Virgin Islands, U.S."),
    ("Wallis and Futuna", "WF", "WLF", "876", "Wallis and Futuna"),
    ("Western Sahara", "EH", "ESH", "732", "Western Sahara"),
    ("Yemen", "YE", "YEM", "887", "Yemen"),
    ("Zambia", "ZM", "ZMB", "894", "Zambia"),
    ("Zimbabwe", "ZW", "ZWE", "716", "Zimbabwe")]

# U.S. State  List

states=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

#U.S. State Postal Abbreviations List

states1 = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# Striping messy tags

def strip_tags(html):
    s =  html.replace("</italic>"," ")
    s =  html.replace ("<country>", " ")
    s =  html.replace ("</country>", " ")
    s =  html.replace ("country", " ")
    s =  s.replace("<italic>"," ")
    s =  s.replace("<sub>"," ")
    s =  s.replace("</sub>"," ")
    s =  s.replace("<p>"," ")
    s =  s.replace("</p>"," ")
    s =  s.replace("<sup>"," ")
    s =  s.replace("</sup>"," ")
    s =  s.replace("<title>"," ")
    s =  s.replace("</title>"," ")
    s =  s.replace("<bold>"," ")
    s =  s.replace("</bold>"," ")
    s =  s.replace("<article-title>"," ")
    s =  s.replace("</alt-title>"," ")
    s =  s.replace("</article-title>"," ")
    s =  s.replace("<alt-title>"," ")
    s=   s.replace("/", " ")
    s=   s.replace("<", " ")
    s=   s.replace(">", " ")
    s=   s.replace("sec", " ")
    s=   s.replace("bold", " ")
    s=   s.replace("aff id=", " ")
    s=   s.replace("label", " ")
    s=   s.replace("subtitle", " ")
    s=   s.replace("p id=", " ")
    s=   s.replace("aff", " ")
 
    return s 

# Striping numbers and other useless characters for further processing

def onlywords(text1):
   text1=re.sub(r'[0-9]+', ', ', text1)
   pattern = r"[a-zA-Z,]+"
   result = re.findall(pattern,text1)
   division=" "
   strg=division.join(result)
   return strg


# Function for extracting countries from afflitions

def  count_ex(texts):
           #print ( "text=",texts)
           texts=onlywords(texts)
           for cn in countries:
                if texts.endswith(cn[0]):
                    #print(cn[0])
                    country_final=cn[2]
                    if country_final=="GEO" :
                         if texts.find("Atlanta")!=-1 or  texts.find("Columbus")!=-1 :country_final="USA"

                    return country_final
           doc = nlp(texts)
           country_final=""
           if texts.find(",USA")!=-1 or texts.find(" USA")!=-1:
             country_final="USA"
             return country_final


           for st in states1:
               st_name=","+" "+st+" "
               if texts.find(st_name)!=-1 or texts.endswith(","+" "+st):
                 country_final="USA"
                 return country_final

            
           if  country_final=="":
            for entity in doc.ents:
                #print(entity.text, "=", entity.label_)
                if  entity.label_=="GPE":
                     #print(entity.text, "=", entity.label_)
                     count_name=entity.text
                     for cn in countries:
                          if count_name==cn[0]:
                              #print ("***"+ cn[2])
                              country_final=cn[2]
                              if country_final=="GEO" :
                                    # Exception for Georgia, since Georgia is both a name of the country Georgia and one of the US states, one more test should be carried out.
                                  if texts.find("Atlanta")!=-1 or  texts.find("Columbus")!=-1  or texts.find("Savannah")!=-1  or texts.find("Athens")!=-1 :country_final="USA"
                
                              
            if country_final=="":
              #print("phase3")
              for entity in doc.ents:
                if  entity.label_=="GPE":
                  for st in states:
                     if entity.text==st:
                         country_final="USA"
                         #print ("*** phase3 state!!  USA")
                         
            if country_final=="":
              #print("phase2")
              for entity in doc.ents:
                if  entity.label_=="GPE":
                 #print (entity.text)
                 #print ( GeoText(entity.text).country_mentions  )
                 inputc=entity.text.replace(" ",",")
                 mycountry=GeoText(inputc).country_mentions
 
                 for key,values in  mycountry.items():
                     for cn in countries:
                          if key==cn[0] or key==cn[1]:
                              #print ("pahe 2 "+cn[2])
                              country_final=cn[2]

            if country_final=="":
              #print("phase4")
              for entity in doc.ents:
                if  entity.label_=="ORG":
                 inputc=entity.text
                 mycountry=GeoText(inputc).country_mentions
 
                 for key,values in  mycountry.items():
                     for cn in countries:
                          if key==cn[0] or key==cn[1]:
                              #print ("*** phase4 "+cn[2])
                              country_final=cn[2]                            

            #print (country_final)                 
            return country_final
        
# Counting the records number
rec_no=0                              

# Opening the output file extracted.tsv

tableoutput =["Journal ID ","issn No. "," Publisher Name"," Location","Article Title","","Abstract "," Date","Countibuter(s) "," affiliation(s)" ,"", "Countries Involved"]
fw=codecs.open("D:\\Pubmed\\extracted.tsv","w",encoding='utf-8')
fw.write("file_id"+"\t")
for x in tableoutput:
     fw.write(x+"\t")
     x=""
fw.write("\n")

# Accessing the xml files
path = "D:\\uncomm1\\" # path for the articles folders

for rootc, dirs, files in os.walk(path):
     for file in fnmatch.filter(files,"*.nxml"):
        #print(file)
        fileWithPath=str(os.path.join(rootc, file))
        #print(fileWithPath)
        utf8_file= codecs.open(fileWithPath, 'r' ,"utf-8")
        tree = ET.parse(utf8_file)
        root = tree.getroot()

        fw.write(file+"\t")
        
# Extracting  journal-id, issn, publisher-name and publisher-loc from XML header
        for x in tableoutput:
            x=""
     
        i=-1
        for child in root[0][0]:
            i=i+1
            if child.tag=="journal-id":
                    tableoutput[0]= tableoutput[0]+ " " + child.text.rstrip('\r\n\t')
            if child.tag=="issn":
                    tableoutput[1]=child.text.rstrip('\r\n\t')
            for child in root[0][0][i]:
                      #print (child.tag, child.text.rstrip('\r\n\t'))
                      if child.tag=="publisher-name":
                            tableoutput[2]=tableoutput[2]+child.text.rstrip('\r\n\t')
                      if child.tag=="publisher-loc":
                          tableoutput[3]= tableoutput[3]+child.text.rstrip('\r\n\t')
# End of extracting  journal-id ....

# Extracting artcile title
        title=""
        i=-1
        for child in root[0][1]:
             i=i+1
             if child.tag=="title-group":
                 titleindex=i
                 #print ("titleindex:",titleindex)
                 break
        for child in root[0][1][titleindex]:
                   title=title + " "+ str(ET.tostring(child), "utf-8")
                   #title=re.sub(r'\W+\s', '', title)
                   
        tableoutput[4]=title
# End of extracting artcile title
    
# Extracting the article abstract
 
        i=-1
        abstracti=0
        for child in root[0][1]:
             i=i+1
             
             if child.tag=="abstract":
                 abstracti=i
                 #print ("abstract:",abstracti)
                 break
        abstract=""
        if abstracti!=0 :
             for child in root[0][1][abstracti]:
                   abstract=abstract + " "+str(ET.tostring(child), "utf-8")
                   #abstract=re.sub(r'\W+\s', '', abstract)

        tableoutput[6]= (abstract)
# End of extracting the article abstract
        
# Extracting the date
 
        i=-1
        for child in root[0][1]:
             i=i+1
             
             if child.tag=="pub-date":
                 pubi=i
                 #print ("pubi:",pubi)
                 break
        pubdate="";
        for child in root[0][1][pubi]:
                   #print (child.tag, child.text)
                   pubdate=child.text.rstrip('\r\n\t')
                   if pubdate==None:
                       pubdate="NA"
        #print (pubdate)
        tableoutput[7]=pubdate
# End of extracting the date

# Extracting  the artcile contributers
        i=-1
        cgi=-1
        for child in root[0][1]:
             i=i+1
        
             if child.tag=="contrib-group":
                 cgi=i
                 #print ("cgi:",cgi)           
        i=-1
        all_contrib=[]
        if cgi!=-1: 
         for x in root[0][1][cgi]:
            i=i+1
            for child in root[0][1][cgi][i]:
                if child.tag=="name":
                    for child in root[0][1][cgi][i][0]:
                        #print (child.tag, child.text)
                        if child.tag=="surname":
                            surname=child.text
                        if child.tag=="given-names":
                           if child.text!=None:
                               all_contrib.append(surname+" "+child.text)
        tableoutput[8]=str(all_contrib)
        country_final=""
# End of extracting  the artcile contributers  

# Extracting the artcicle affiliations and involved_countries
        all_aff=""
        involved_countries=""
 
        for el in root[0][1][cgi].iterfind('aff'):
            aff=str(ET.tostring(el), "UTF-8")
            aff=strip_tags(aff)
 
            all_aff=all_aff+""+aff
            involved_countries=involved_countries+" "+  str( count_ex(str(aff)))
        tableoutput[9]=all_aff
        tableoutput[11]=involved_countries
        tableoutput[10]=""
# End of extracting the artcicle affiliations and involved_countries

# Extracting the affiliations and involved_countries when affilition is in xml root
        if all_aff=="":
  
         involved_countries=""
 
         for el in root[0][1].iterfind('aff'):
             aff=str(ET.tostring(el), "UTF-8")
            
             aff=strip_tags(aff)
   
             all_aff=all_aff+"."+aff
             involved_countries=involved_countries+" "+  str( count_ex(str(aff)))
    
         tableoutput[9]=all_aff
         tableoutput[11]=involved_countries
# End of extracting the affiliations

# Extracting country from contributers attributes(not affiliations)
        if tableoutput[11]=="":
         involved_countries=""
         contrib1=root[0][1][cgi].iterfind('contrib')
         for el in contrib1:
            cont=str(ET.tostring(el), "utf-8")
            cont=strip_tags(cont)
            doc=nlp(cont)
            for entity in doc.ents:
                #print(entity.text, "=", entity.label_)
                if  entity.label_=="GPE":
                     #print(entity.text, "=", entity.label_)
                     count_name=entity.text
                     for cn in countries:
                          if count_name==cn[0]:
                              #print ("*** from names "+ cn[2])
                              country_final=cn[2]
            involved_countries=involved_countries+" "+  country_final
         tableoutput[11]=involved_countries

#Writing the list to file
        
        for x in range(0, 12):
                  tableoutput[x]=tableoutput[x].replace("\\n"," ")
                  tableoutput[x]=tableoutput[x].replace("\n"," ")
                  tableoutput[x]=strip_tags(tableoutput[x])
                  fw.write(tableoutput[x]+"\t")
                
        fw.write("\n")
        rec_no=rec_no+1
        #print(rec_no)
                      
fw.close()
